# Document Vector Embeddings

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

/Users/altaf/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [3]:
raw_documents = TextLoader('state_of_the_union.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)


In [27]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(documents, embeddings)

In [31]:
query = "What did the president say about ukraine"
docs = db.similarity_search(query)



4


In [ ]:
embedding_vector = embeddings.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)

display(embedding_vector)

In [20]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff" , verbose= True)

In [25]:
query = "What are the challenges facing voting rights? And what actions are proposed deal with these challenges? And how will these actions work?"
docs = db.similarity_search(query)

chain.run(input_documents=docs, question=query)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

And I will keep doing everything in my power to crack down on gun trafficking and ghost guns you can buy online and make at home—they have no serial numbers and can’t be traced. 

And I ask Congress to pass proven measures to reduce gun violence. Pass universal background checks. Why should anyone on a terrorist list be able to purchase a weapon? 

Ban assault weapons and high-capacity magazines. 

Repeal the liability shield that makes gun manufacturers the only industry in America that can’t be sued. 

These laws don’t infringe on the Second Amendment. They save lives. 

The most fundamental right in America is the right to vote – and to have it counted. And it’s under assault. 

In state after state, new laws have been passed, not only to suppr

' The challenge facing voting rights is that new laws have been passed in many states to suppress the vote and subvert entire elections. To deal with this challenge, the President has called on the Senate to pass the Freedom to Vote Act, the John Lewis Voting Rights Act, and the Disclose Act. The Freedom to Vote Act would make it easier for people to register and vote, the John Lewis Voting Rights Act would restore key provisions of the Voting Rights Act of 1965, and the Disclose Act would require organizations that fund political campaigns to disclose their donors.'